In [12]:
import torch
import torchvision.transforms as transforms
import numpy as np
import torchvision
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from numpy.random import RandomState
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Subset

In [13]:
mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]

# Define transformations for training and validation data
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10), 
    transforms.RandomCrop(32, padding=4),  
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

In [14]:
num_classes = 2
num_samples_per_class = 25

cifar_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
random_classes = np.random.choice(range(10), num_classes, replace=False)
selected_indices_train = []
selected_indices_val = []


for i in random_classes:
    class_indices = np.where(np.array(cifar_data.targets) == i)[0]
    selected_indices_train.extend(np.random.choice(class_indices, num_samples_per_class, replace=False))
    remaining_indices = list(set(class_indices) - set(selected_indices_train))
    selected_indices_val.extend(remaining_indices)

Files already downloaded and verified


In [15]:
train_data = Subset(cifar_data, selected_indices_train)
val_data = Subset(cifar_data, selected_indices_val)

# Create DataLoader for training and validation datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=len(train_data), shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=len(val_data), shuffle=False)

# Extract features and labels from training and validation datasets
train_features, train_labels = next(iter(train_loader))
val_features, val_labels = next(iter(val_loader))

train_features = train_features.view(train_features.size(0), -1).numpy()
val_features = val_features.view(val_features.size(0), -1).numpy()

# Normalize features using StandardScaler
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)


In [16]:
model = SVC(kernel='rbf', C=1.0, gamma='auto')  # Adjust gamma value
model.fit(train_features_scaled, train_labels)

# Predict on validation set
val_predictions = model.predict(val_features_scaled)

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print("Validation Accuracy: {:.2f}%".format(accuracy * 100))

Validation Accuracy: 74.08%


In [18]:
hyperparameters = [
    {'C': 1.0, 'gamma': 'scale', 'random_seed': 1},
    {'C': 0.5, 'gamma': 'scale', 'random_seed': 2},
    {'C': 1.5, 'gamma': 'scale', 'random_seed': 1},
    {'C': 0.5, 'gamma': 'auto', 'random_seed': 2},
    {'C': 0.5, 'gamma': 'auto', 'random_seed': 1},
    {'C': 0.5, 'gamma': 'auto', 'random_seed': 2},
]

In [19]:
svm_models = []
for params in hyperparameters:
    # Initialize SVM model with specified hyperparameters
    model = SVC(kernel='rbf', C=params['C'], gamma=params['gamma'], random_state=params['random_seed'])
    
    # Train model
    model.fit(train_features_scaled, train_labels)
    
    # Add trained model to list
    svm_models.append(model)

In [21]:
individual_predictions = []
for model in svm_models:
    predictions = model.predict(val_features_scaled)
    individual_predictions.append(predictions)

ensemble_predictions_avg_discrete = np.round(ensemble_predictions_avg)
ensemble_predictions_vote_discrete = np.round(ensemble_predictions_vote)

accuracy_avg_discrete = accuracy_score(val_labels, ensemble_predictions_avg_discrete)
accuracy_vote_discrete = accuracy_score(val_labels, ensemble_predictions_vote_discrete)

print("Ensemble Accuracy (Averaging) after conversion: {:.2f}%".format(accuracy_avg_discrete * 100))
print("Ensemble Accuracy (Voting) after conversion: {:.2f}%".format(accuracy_vote_discrete * 100))

Ensemble Accuracy (Averaging) after conversion: 69.11%
Ensemble Accuracy (Voting) after conversion: 69.11%
